In [1]:
import pandas as pd
data = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn2.csv")
data.head()

,cDNA,Amino acid (HGVS),Amino acid (Legacy),pos,Type,Domain,Effect,Sequence Context,Original AA,New AA,AminoBefore,AminoAfter,Locationingene,nitroBaseBef,nitroBaseAft,Severity
0,-1.411869,-1.412448,-1.412429,2.224128,0,0,0,186,6,3,3,3,11,0,1,0
1,-1.499705,-1.499467,-1.500672,-0.958553,0,6,0,90,12,20,10,18,0,0,2,2
2,-1.499705,-1.499467,-1.500672,-0.958553,0,6,0,90,12,20,10,18,0,0,2,2
3,-1.499705,-1.499467,-1.500672,-0.958553,0,6,0,90,12,20,10,18,0,0,2,2
4,-1.499705,-1.499467,-1.500672,-0.958553,0,6,0,90,12,20,10,18,0,0,2,2


## without K fold

In [5]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load your dataset
df = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn2.csv")

# Prepare data
X = df[['cDNA', 'Amino acid (HGVS)', 'Amino acid (Legacy)', 'pos', 'Type', 'Domain', 'Effect', 'Sequence Context', 'Original AA', 'New AA', 'AminoBefore', 'AminoAfter', 'Locationingene', 'nitroBaseBef', 'nitroBaseAft']]
y = df['Severity']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape data for RNN: (samples, timesteps, features)
# Here, timesteps will be 1 because we're treating each sample as a single timestep
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# Define model
model = Sequential()
model.add(SimpleRNN(64, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 3 classes for Severity

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=1, validation_split=0.2)

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')


Epoch 1/10
5332/5332 [==============================] - 13s 2ms/step - loss: 0.9669 - accuracy: 0.5051 - val_loss: 0.9065 - val_accuracy: 0.5529
Epoch 2/10
5332/5332 [==============================] - 13s 2ms/step - loss: 0.9118 - accuracy: 0.5521 - val_loss: 0.8909 - val_accuracy: 0.5596
Epoch 3/10
5332/5332 [==============================] - 13s 2ms/step - loss: 0.8884 - accuracy: 0.5696 - val_loss: 0.8695 - val_accuracy: 0.5769
Epoch 4/10
5332/5332 [==============================] - 13s 2ms/step - loss: 0.8706 - accuracy: 0.5848 - val_loss: 0.8955 - val_accuracy: 0.5626
Epoch 5/10
5332/5332 [==============================] - 13s 2ms/step - loss: 0.8569 - accuracy: 0.5955 - val_loss: 0.8738 - val_accuracy: 0.5986
Epoch 6/10
5332/5332 [==============================] - 14s 3ms/step - loss: 0.8395 - accuracy: 0.6092 - val_loss: 0.8594 - val_accuracy: 0.5994
Epoch 7/10
5332/5332 [==============================] - 13s 3ms/step - loss: 0.8301 - accuracy: 0.6131 - val_loss: 0.8447 - val_ac

## With K fold

In [6]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Load your dataset
df = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn2.csv")

# Prepare data
X = df[['cDNA', 'Amino acid (HGVS)', 'Amino acid (Legacy)', 'pos', 'Type', 'Domain', 'Effect', 'Sequence Context', 'Original AA', 'New AA', 'AminoBefore', 'AminoAfter', 'Locationingene', 'nitroBaseBef', 'nitroBaseAft']]
y = df['Severity']

# Standardize features (optional, but recommended for RNNs)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape data for RNN: (samples, timesteps, features)
# Here, timesteps will be 1 because we're treating each sample as a single timestep
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Define k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_index, test_index in kf.split(X_reshaped):
    X_train, X_test = X_reshaped[train_index], X_reshaped[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Define model
    model = Sequential()
    model.add(SimpleRNN(64, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # 3 classes for Severity

    # Compile model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train model
    model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=0, validation_split=0.2)
    
    # Evaluate model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    accuracies.append(accuracy)

print(f'Average Accuracy: {np.mean(accuracies)}')


Average Accuracy: 0.5510060787200928


In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load your data
df = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn2.csv")

# Separate features and labels
X = df.drop(columns=['Severity'])
y = df['Severity']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_accuracies = []

fold = 1
for train_index, val_index in kf.split(X_scaled):
    print(f"Processing fold {fold}")
    
    # Split data
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y_one_hot[train_index], y_one_hot[val_index]
    
    # Reshape for RNN
    X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val_rnn = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
    
    # Define the RNN model
    model = Sequential()
    model.add(SimpleRNN(units=50, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=False))
    model.add(Dense(3, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Train the model
    history = model.fit(X_train_rnn, y_train, epochs=20, batch_size=64, validation_data=(X_val_rnn, y_val),
                        callbacks=[early_stopping, reduce_lr], verbose=1)
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_val_rnn, y_val)
    print(f'Fold {fold} - Loss: {loss}, Accuracy: {accuracy}')
    
    # Store the accuracy of this fold
    fold_accuracies.append(accuracy)
    
    # Increment fold counter
    fold += 1

    # Clear the model from memory
    K.clear_session()

# Calculate average accuracy
average_accuracy = np.mean(fold_accuracies)
print(f'Average Accuracy across all folds: {average_accuracy}')


Processing fold 1
Epoch 1/20
105/105 [==============================] - 2s 5ms/step - loss: 1.0545 - accuracy: 0.4509 - val_loss: 1.0279 - val_accuracy: 0.4721 - lr: 0.0010
Epoch 2/20
105/105 [==============================] - 0s 3ms/step - loss: 1.0126 - accuracy: 0.4863 - val_loss: 1.0087 - val_accuracy: 0.4667 - lr: 0.0010
Epoch 3/20
105/105 [==============================] - 0s 3ms/step - loss: 0.9998 - accuracy: 0.4924 - val_loss: 0.9967 - val_accuracy: 0.4817 - lr: 0.0010
Epoch 4/20
105/105 [==============================] - 0s 3ms/step - loss: 0.9884 - accuracy: 0.5089 - val_loss: 0.9891 - val_accuracy: 0.5021 - lr: 0.0010
Epoch 5/20
105/105 [==============================] - 0s 4ms/step - loss: 0.9785 - accuracy: 0.5200 - val_loss: 0.9847 - val_accuracy: 0.5147 - lr: 0.0010
Epoch 6/20
105/105 [==============================] - 0s 3ms/step - loss: 0.9692 - accuracy: 0.5194 - val_loss: 0.9765 - val_accuracy: 0.5129 - lr: 0.0010
Epoch 7/20
105/105 [==============================] 

In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load your data
df = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn2.csv")

# Separate features and labels
X = df.drop(columns=['Severity'])
y = df['Severity']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_accuracies = []

fold = 1
for train_index, val_index in kf.split(X_scaled):
    print(f"Processing fold {fold}")
    
    # Split data
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y_one_hot[train_index], y_one_hot[val_index]
    
    # Reshape for RNN
    X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val_rnn = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
    
    # Define the RNN model
    model = Sequential()
    model.add(SimpleRNN(units=100, activation='tanh', input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(3, activation='softmax'))

    # Compile the model with a lower learning rate
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

    # Train the model
    history = model.fit(X_train_rnn, y_train, epochs=30, batch_size=32, validation_data=(X_val_rnn, y_val),
                        callbacks=[early_stopping, reduce_lr], verbose=1)
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_val_rnn, y_val)
    print(f'Fold {fold} - Loss: {loss}, Accuracy: {accuracy}')
    
    # Store the accuracy of this fold
    fold_accuracies.append(accuracy)
    
    # Increment fold counter
    fold += 1

    # Clear the model from memory
    K.clear_session()

# Calculate average accuracy
average_accuracy = np.mean(fold_accuracies)
print(f'Average Accuracy across all folds: {average_accuracy}')


Processing fold 1
Epoch 1/30
209/209 [==============================] - 2s 5ms/step - loss: 1.0284 - accuracy: 0.4614 - val_loss: 0.9695 - val_accuracy: 0.5003 - lr: 0.0010
Epoch 2/30
209/209 [==============================] - 1s 3ms/step - loss: 0.9582 - accuracy: 0.5175 - val_loss: 0.9581 - val_accuracy: 0.5195 - lr: 0.0010
Epoch 3/30
209/209 [==============================] - 1s 3ms/step - loss: 0.9454 - accuracy: 0.5296 - val_loss: 0.9402 - val_accuracy: 0.5117 - lr: 0.0010
Epoch 4/30
209/209 [==============================] - 1s 3ms/step - loss: 0.9301 - accuracy: 0.5386 - val_loss: 0.9399 - val_accuracy: 0.5237 - lr: 0.0010
Epoch 5/30
209/209 [==============================] - 1s 3ms/step - loss: 0.9239 - accuracy: 0.5427 - val_loss: 0.9387 - val_accuracy: 0.5189 - lr: 0.0010
Epoch 6/30
209/209 [==============================] - 1s 3ms/step - loss: 0.9177 - accuracy: 0.5443 - val_loss: 0.9257 - val_accuracy: 0.5171 - lr: 0.0010
Epoch 7/30
209/209 [==============================] 

## With Stratified K fold

In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load your data
df = pd.read_csv(r"D:\Github portfolio\Hemophilia\Hemophilia-Severity-Predictor\datasets\resampled_data_adasyn2.csv")

# Separate features and labels
X = df.drop(columns=['Severity'])
y = df['Severity']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

# Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_accuracies = []

fold = 1
for train_index, val_index in skf.split(X_scaled, y_encoded):
    print(f"Processing fold {fold}")
    
    # Split data
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y_one_hot[train_index], y_one_hot[val_index]
    
    # Reshape for RNN
    X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val_rnn = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
    
    # Define the RNN model
    model = Sequential()
    model.add(SimpleRNN(units=128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(SimpleRNN(units=64, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Train the model
    history = model.fit(X_train_rnn, y_train, epochs=50, batch_size=16, validation_data=(X_val_rnn, y_val),
                        callbacks=[early_stopping, reduce_lr], verbose=1)
    
    # Evaluate the model
    loss, accuracy = model.evaluate(X_val_rnn, y_val)
    print(f'Fold {fold} - Loss: {loss}, Accuracy: {accuracy}')
    
    # Store the accuracy of this fold
    fold_accuracies.append(accuracy)
    
    # Increment fold counter
    fold += 1

    # Clear the model from memory
    K.clear_session()

# Calculate average accuracy
average_accuracy = np.mean(fold_accuracies)
print(f'Average Accuracy across all folds: {average_accuracy}')


Processing fold 1
Epoch 1/50
417/417 [==============================] - 4s 5ms/step - loss: 1.0226 - accuracy: 0.4606 - val_loss: 0.9464 - val_accuracy: 0.5345 - lr: 0.0010
Epoch 2/50
417/417 [==============================] - 2s 4ms/step - loss: 0.9707 - accuracy: 0.4989 - val_loss: 0.9276 - val_accuracy: 0.5297 - lr: 0.0010
Epoch 3/50
417/417 [==============================] - 1s 4ms/step - loss: 0.9531 - accuracy: 0.5146 - val_loss: 0.9172 - val_accuracy: 0.5393 - lr: 0.0010
Epoch 4/50
417/417 [==============================] - 1s 3ms/step - loss: 0.9464 - accuracy: 0.5226 - val_loss: 0.9131 - val_accuracy: 0.5489 - lr: 0.0010
Epoch 5/50
417/417 [==============================] - 1s 4ms/step - loss: 0.9381 - accuracy: 0.5229 - val_loss: 0.9054 - val_accuracy: 0.5375 - lr: 0.0010
Epoch 6/50
417/417 [==============================] - 2s 4ms/step - loss: 0.9284 - accuracy: 0.5347 - val_loss: 0.9047 - val_accuracy: 0.5537 - lr: 0.0010
Epoch 7/50
417/417 [==============================] 